In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Add as add
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from numpy import array

In [24]:
df = pd.read_excel('tamil.xlsx')

In [25]:
df.head()

,Unnamed: 0,படம்,தலைப்பு
0,0,1000268201_693b08cb0e.jpg,இளஞ்சிவப்பு நிற உடையில் ஒரு குழந்தை நுழையும் ...
1,1,1000268201_693b08cb0e.jpg,ஒரு பெண் ஒரு மர கட்டிடத்திற்குள் செல்கிறாள்.
2,2,1000268201_693b08cb0e.jpg,ஒரு சிறிய பெண் ஒரு மர விளையாட்டு இல்லத்தில் ஏ...
3,3,1000268201_693b08cb0e.jpg,ஒரு சிறுமி தன் விளையாட்டு விடுதிக்கு படிக்கட்...
4,4,1000268201_693b08cb0e.jpg,இளஞ்சிவப்பு நிற உடையில் ஒரு சிறுமி மர அறைக்கு...


In [26]:
x = df.iloc[:,2]
x

0        இளஞ்சிவப்பு நிற உடையில் ஒரு குழந்தை நுழையும் ...
1            ஒரு பெண் ஒரு மர கட்டிடத்திற்குள் செல்கிறாள்.
2        ஒரு சிறிய பெண் ஒரு மர விளையாட்டு இல்லத்தில் ஏ...
3        ஒரு சிறுமி தன் விளையாட்டு விடுதிக்கு படிக்கட்...
4        இளஞ்சிவப்பு நிற உடையில் ஒரு சிறுமி மர அறைக்கு...
                              ...                        
4994     ஒரு மலையின் முன் நிற்கும் போது மனிதன் பச்சை ந...
4995     ஒரு பிரவுன் சோவ் கலவை நாய் ஒரு குழப்பமான முற்...
4996     பழுப்பு நிற நாய் ஒரு சேற்று முற்றத்தில் நிற்க...
4997     ஒரு பழுப்பு நிற நாய் ஒரு பச்சை நிற வேன் மற்று...
4998    ஒரு பெரிய பழுப்பு நாய் ஒரு வீட்டின் முற்றத்தில...
Name:  தலைப்பு, Length: 4999, dtype: object

In [50]:
d = {}
d1 = {}
c = 1
t = ""
for i in x:
    t = t + i
k=t.replace("\n", "")
k=k.replace("\u200c", "")
text = k
d["start"] = c
c = c + 1
d['end'] = c
c = c + 1
for i in text.split() :
    if i not in d:
        d[i] = c
        d1[c] = i
        c = c + 1
d,d1

({'start': 1,
  'end': 2,
  'இளஞ்சிவப்பு': 3,
  'நிற': 4,
  'உடையில்': 5,
  'ஒரு': 6,
  'குழந்தை': 7,
  'நுழையும்': 8,
  'வழியில்': 9,
  'படிக்கட்டுகளில்': 10,
  'ஏறுகிறது.': 11,
  'பெண்': 12,
  'மர': 13,
  'கட்டிடத்திற்குள்': 14,
  'செல்கிறாள்.': 15,
  'சிறிய': 16,
  'விளையாட்டு': 17,
  'இல்லத்தில்': 18,
  'ஏறுகிறாள்.': 19,
  'சிறுமி': 20,
  'தன்': 21,
  'விடுதிக்கு': 22,
  'அறைக்குள்': 23,
  'கருப்பு': 24,
  'நாய்': 25,
  'மற்றும்': 26,
  'புள்ளி': 27,
  'சண்டையிடுகின்றன': 28,
  'நாயும்,': 29,
  'மூன்று': 30,
  'நாயும்': 31,
  'சாலையில்': 32,
  'ஒன்றோடு': 33,
  'ஒன்று': 34,
  'விளையாடிக்': 35,
  'கொண்டிருந்தன.': 36,
  'பழுப்பு': 37,
  'நிறப்': 38,
  'புள்ளிகளுடன்': 39,
  'வெள்ளை': 40,
  'தெருவில்': 41,
  'ஒருவரையொருவர்': 42,
  'பார்த்துக்': 43,
  'கொண்டிருக்கின்றன.': 44,
  'வெவ்வேறு': 45,
  'இனத்தைச்': 46,
  'சேர்ந்த': 47,
  'இரண்டு': 48,
  'நாய்கள்': 49,
  'ஒன்றையொன்று': 50,
  'பார்த்துக்கொண்டிருக்கின்றன.': 51,
  'நடைபாதையில்': 52,
  'நோக்கி': 53,
  'நகர்கின்றன.வர்ணம்': 54,
  'பூசப்

In [28]:
len(d)

6654

In [29]:
x_train = []
r = 0
for i in x :
    t = [d["start"]]
    k=i.replace("\n", "")
    k=k.replace("\u200c", "")
    for j in k.split():
        try :
            t.append(d[j])
        except:
            continue
    t.append(d["end"])
    x_train.append(t)
x_train

[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 2],
 [1, 6, 12, 6, 13, 14, 15, 2],
 [1, 6, 16, 12, 6, 13, 17, 18, 19, 2],
 [1, 6, 20, 21, 17, 22, 10, 19, 2],
 [1, 3, 4, 5, 6, 20, 13, 23, 15, 2],
 [1, 6, 24, 25, 26, 6, 27, 25, 28, 2],
 [1, 6, 24, 29, 30, 4, 31, 32, 33, 34, 35, 36, 2],
 [1, 6, 24, 31, 37, 38, 39, 6, 40, 31, 41, 42, 43, 44, 2],
 [1, 45, 46, 47, 48, 49, 32, 50, 51, 2],
 [1, 52, 48, 49, 50, 53, 2],
 [1, 59, 55, 6, 20, 6, 56, 57, 58, 59, 55, 60, 61, 62, 2],
 [1, 6, 63, 59, 55, 60, 61, 6, 20, 62, 2],
 [1, 64, 6, 16, 12, 65, 40, 66, 61, 67, 68, 2],
 [1, 69, 70, 61, 71, 72, 6, 12, 62, 2],
 [1, 73, 64, 74, 75, 76, 77, 2],
 [1, 6, 78, 6, 79, 80, 81, 82, 25, 83, 84, 2],
 [1, 6, 40, 25, 85, 79, 6, 78, 80, 86, 2],
 [1, 6, 78, 87, 6, 79, 88, 89, 90, 6, 40, 26, 24, 25, 84, 2],
 [1, 6, 91, 92, 78, 93, 94, 95, 79, 80, 86, 2],
 [1, 96, 97, 98, 99, 100, 79, 101, 78, 2],
 [1, 102, 4, 103, 104, 6, 78, 105, 106, 107, 2],
 [1, 6, 78, 102, 103, 26, 108, 109, 2],
 [1, 110, 26, 111, 6, 78, 112, 113, 109, 2],


In [30]:
im = pd.read_json('image_vectors.json')
im.head(10)

,image_name,vector
0,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
1,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
2,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
3,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
4,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
5,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
6,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
7,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
8,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
9,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."


In [31]:
x_im = im['vector']

In [32]:
len(x_im),len(x_train)

(4999, 4999)

In [40]:
c = 0
max_length = 100
vocaab_size = 6654
epoch = 100
number_pics_per_batch = 3
X1 = []
X2 = []
y = []
steps = len(text)//number_pics_per_batch
for seq in x_train:
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocaab_size)[0]
        X1.append(x_im[c])
        X2.append(in_seq)
        y.append(out_seq)
    c = c + 1

In [12]:
def data_generator(x_train):
    c = 0
    max_length = 100
    vocaab_size = 6205
    epoch = 100
    number_pics_per_batch = 3
    X1 = []
    X2 = []
    y = []
    n = 0
    steps = len(text)//number_pics_per_batch
    while 1 :
        n = n + 1
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocaab_size)[0]
            X1.append(x_im[c])
            X2.append(in_seq)
            y.append(out_seq)
        c = c + 1
    # yield the batch data
        if n==number_pics_per_batch:
            yield [[array(X1), array(X2)], array(y)]
            X1, X2, y = list(), list(), list()
            n=0
            c = 0

In [44]:
embedding_dim = 200
inputs1=Input(shape=(2048,))
f1=Dropout(0.5)(inputs1)
f2=Dense(256,activation='relu')(f1)
inputs2=Input(shape=(max_length,))
se1 = Embedding(vocaab_size, embedding_dim, mask_zero=True)(inputs2)
se2=Dropout(0.5)(se1)
se3=LSTM(256)(se2)
decoder1=add()([f2,se3])
decoder2=Dense(256,activation='relu')(decoder1)
d1=Dropout(0.5)(decoder2)

decoder3=Dense(256,activation='relu')(d1)
d2=Dropout(0.5)(decoder3)

outputs=Dense(vocaab_size,activation='softmax')(d2)
model=Model(inputs=[inputs1,inputs2],outputs=outputs)



In [45]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 100, 200)     1330800     ['input_4[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 2048)         0           ['input_3[0][0]']                
                                                                                            

In [46]:
adam=optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam)

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [47]:
model.fit([np.array(x_train_img),np.array(x_train_seq)],y_train_seq, epochs=5, batch_size=512, verbose=1,workers=-1)

Epoch 1/5
91/91 [==============================] - 365s 4s/step - loss: 14.7938
Epoch 2/5
91/91 [==============================] - 413s 5s/step - loss: 8.3254
Epoch 3/5
91/91 [==============================] - 424s 5s/step - loss: 7.6074
Epoch 4/5
91/91 [==============================] - 420s 5s/step - loss: 7.2821
Epoch 5/5
37/91 [===========>..................] - ETA: 4:21 - loss: 7.1445

KeyboardInterrupt: 

In [51]:
o=model.predict([np.expand_dims(x_train_img[0],axis=0),np.expand_dims(x_train_seq[0],axis=0)])
dic ={}
for i in range(len(o[0])):
    dic[i+1] = o[0][i]
dic={k: v for k, v in sorted(dic.items(), key=lambda item: item[1],reverse=True)}
d1[list(dic.keys())[0]],list(dic.keys())[0],np.argmax(y_train_seq[0])

('இளஞ்சிவப்பு', 3, 3)

In [48]:
model.save('tamil_model.h5')

In [55]:
import stanza

In [58]:
nlp = stanza.Pipeline(lang='te', processors='tokenize,ner')

2022-04-26 11:38:51 INFO: Loading these models for language: te (Telugu):
| Processor | Package |
-----------------------
| tokenize  | mtg     |
| ner       | default |

2022-04-26 11:38:51 INFO: Use device: cpu
2022-04-26 11:38:51 INFO: Loading: tokenize
2022-04-26 11:38:51 INFO: Loading: ner
2022-04-26 11:38:51 ERROR: Cannot load model from C:\Users\HP\stanza_resources\te\ner\default.pt


UnsupportedProcessorError: Processor ner is not known for language te.  If you have created your own model, please specify the ner_model_path parameter when creating the pipeline.

In [18]:
np.array(y).shape

(45927, 6205)

In [19]:
np.array(X1).shape

(45927, 2048)

In [20]:
np.array(X2).shape

(45927, 100)

In [18]:
x_train[0]

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]

In [19]:
cur_inp =[]
for i in range(len(x_train[0])-1):
    cur_inp.append(x_train[0][i])
    out = x_train[0][i+1]
    print(cur_inp , out)

[1] 3
[1, 3] 4
[1, 3, 4] 5
[1, 3, 4, 5] 6
[1, 3, 4, 5, 6] 7
[1, 3, 4, 5, 6, 7] 8
[1, 3, 4, 5, 6, 7, 8] 9
[1, 3, 4, 5, 6, 7, 8, 9] 10
[1, 3, 4, 5, 6, 7, 8, 9, 10] 11
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11] 12
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 2


In [41]:
x_train_seq =[]
x_train_img =[]
y_train_seq =[]
for j in range(len(x_train)):
    cur_inp =[]
    for i in range(len(x_train[j])-1):
        cur_inp.append(x_train[j][i])
        out = x_train[j][i+1]
        x_train_seq.append(cur_inp.copy())
        y_train_seq.append(out)
        x_train_img.append(x_im[j].copy())
    print(j,end='\r')

In [42]:
x_train_seq=pad_sequences(x_train_seq, maxlen=max_length)

In [43]:
y_train_seq=to_categorical(y_train_seq, num_classes=vocaab_size)